In [49]:
import pandas as pd
import os
import requests
import numpy as np 
import io
import tqdm

print('Current working directory:', os.getcwd())

Current working directory: /home/prichter/Documents/find-a-bug/sandbox


In [9]:
LOG_DIR = os.path.join(os.getcwd(), '../scripts/log/')

# Running into errors uploading certain annotations to the MariaDB database. 
failures_df = []
for file_name in os.listdir(LOG_DIR):
    failures_df.append(pd.read_csv(os.path.join(LOG_DIR, file_name), comment='#'))
failures_df = pd.concat(failures_df)

/tmp/ipykernel_233074/888517050.py:6: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  failures_df.append(pd.read_csv(os.path.join(LOG_DIR, file_name), comment='#'))
/tmp/ipykernel_233074/888517050.py:6: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  failures_df.append(pd.read_csv(os.path.join(LOG_DIR, file_name), comment='#'))


In [12]:
print(len(failures_df.genome_id.unique()), 'genome IDs failed to upload.')
# Perhaps I should first make sure the files are actually present in the 

1100 genome IDs failed to upload.


In [17]:
failing_genome_ids = failures_df['genome_id'].unique()
np.savetxt('failing_genome_ids.txt', failing_genome_ids, fmt='%s')

In [48]:
gene_ids_in_database_df = []
for genome_id in failing_genome_ids:
    url = f'http://microbes.gps.caltech.edu:8000/get/proteins_r207?genome_id[eq]{genome_id}'
    result = requests.get(url).text 
    try:
        gene_ids_in_database_df.append(pd.read_csv(io.StringIO(result)))
    except:
        print(f'There doesnt seem to be any results for {genome_id}. Length of results:', len(result))
gene_ids_in_database_df = pd.concat(gene_ids_in_database_df)
gene_ids_in_database_df.set_index('gene_id').to_csv('gene_ids_in_database.csv')

    

There doesnt seem to be any results for GCA_011364235.1. Length of results: 0
There doesnt seem to be any results for GCF_015245355.1. Length of results: 0
There doesnt seem to be any results for GCA_011364155.1. Length of results: 0
There doesnt seem to be any results for GCA_011364265.1. Length of results: 0
There doesnt seem to be any results for GCF_015244745.1. Length of results: 0
There doesnt seem to be any results for GCA_011364225.1. Length of results: 0
There doesnt seem to be any results for GCA_011364305.1. Length of results: 0


In [ ]:
# gene_ids_not_in_database_already_downloaded = np.loadtxt('gene_ids_not_in_database.txt', dtype=str)
# gene_ids_to_check = [gene_id for gene_id in failures_df.gene_id.values if (gene_id not in gene_ids_not_in_database_already_downloaded)]
# print(f'Already confirmed {len(gene_ids_not_in_database_already_downloaded)} missing genes.')

gene_ids_not_in_database = []

chunk_size = 50
# Why are there duplicate gene IDs? Are some of the IDs not unique?
gene_ids_to_check = failures_df.gene_id.unique().tolist()
gene_ids_to_check = [gene_ids_to_check[i * chunk_size:(i + 1) * chunk_size] for i in range(len(gene_ids_to_check) // chunk_size + 1)]

pbar = tqdm.tqdm(gene_ids_to_check, desc='Querying Find-A-Bug... Found 0 missing genes.')
for chunk in pbar:
    filter_ = 'gene_id[eq]' + '[or]'.join(chunk)
    url = f'http://microbes.gps.caltech.edu:8000/get/proteins_r207?' + filter_

    result = requests.get(url).text 

    if len(result) == 0:
        print('No results returned!')
        gene_ids_not_in_database += chunk
        pbar.set_description(f'Querying Find-A-Bug... Found {len(gene_ids_not_in_database)} missing genes.')
    else:
        result_df = pd.read_csv(io.StringIO(result))
        if len(result_df) < len(chunk):
            print(f'Retrieved {len(result_df)} results, expected {len(chunk)}.')
            gene_ids_not_in_database += [gene_id for gene_id in chunk if (gene_id not in result_df.gene_id)]
            pbar.set_description(f'Querying Find-A-Bug... Found {len(gene_ids_not_in_database)} missing genes.')


In [69]:
np.savetxt('gene_ids_not_in_database.txt', gene_ids_not_in_database, fmt='%s')

In [70]:
failures_df = failures_df[failures_df.gene_id.isin(gene_ids_not_in_database)]

In [73]:
print('Number of genomes with failed genes:', len(failures_df.genome_id.unique()))

Number of genomes with failed genes: 28


In [77]:
# for genome_id in failures_df.genome_id.unique():
#     print(genome_id)

# "is present" means it is in the source directory (proteins_aa)
# GCF_015244675.1 is present 
# GCF_015244675.1 is present
# GCA_010032545.1 is present
# GCA_011364235.1 is present
# GCA_018401055.1 is present
# GCF_015694425.1 is present

genome_ids = [f"\'{genome_id}\'" for genome_id in failures_df.genome_id.unique()]
print(f"[{', '.join(genome_ids)}]")

['GCF_015244675.1', 'GCA_013140765.1', 'GCA_010032545.1', 'GCA_011364235.1', 'GCA_018401055.1', 'GCF_015694425.1', 'GCF_015245355.1', 'GCA_001779505.1', 'GCA_018608425.1', 'GCA_011364155.1', 'GCA_003157385.1', 'GCF_002909445.1', 'GCA_011364525.1', 'GCA_905182715.1', 'GCA_011364265.1', 'GCA_900545805.1', 'GCF_000709085.1', 'GCF_015244745.1', 'GCA_012518835.1', 'GCA_017467305.1', 'GCF_015265435.1', 'GCF_003946115.1', 'GCA_011364225.1', 'GCA_002313895.1', 'GCF_001675285.1', 'GCA_011364305.1', 'GCF_902499045.1', 'GCA_011364105.1']
